In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#1.1 cohort 테이블 추출(1)
- mimic-iv-hosp-icu.db 와 mimic-iv-ed 데이터를 결합

In [10]:
import pandas as pd
import os
import sqlite3
import numpy as np

# [1] 경로 설정

# ED 데이터셋 폴더 경로
ed_base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/mimic-iv-ed-2.2/'

# HOSP/ICU 데이터베이스 파일 경로
db_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/MIMIC4-hosp-icu.db'

# [2] 데이터 로드 및 코호트 생성
try:
    # 2-1. ED 데이터 로드 (CSV)
    edstays_csv_path = os.path.join(ed_base_path, 'edstays.csv')
    df_edstays = pd.read_csv(edstays_csv_path,
                             dtype={'subject_id': int, 'hadm_id': 'Int64', 'stay_id': int})
    print(f"edstays.csv 로드 완료 (Shape: {df_edstays.shape})")

    # 2-2. HOSP 데이터 로드 (SQLite)
    conn = sqlite3.connect(db_path) # DB 연결
    query = "SELECT subject_id, hadm_id, icd_code, icd_version FROM diagnoses_icd"
    df_diagnoses_icd = pd.read_sql_query(query, conn)
    conn.close() # DB 연결 해제

    df_diagnoses_icd['icd_code'] = df_diagnoses_icd['icd_code'].astype(str)
    print(f"diagnoses_icd DB 테이블 로드 완료 (Shape: {df_diagnoses_icd.shape})")

    # [3] STEMI 진단 코드 정의 및 필터링
    # ICD-10
    stemi_icd10 = ['I210', 'I211', 'I212', 'I213']
    # ICD-9
    stemi_icd9 = ['4100', '4101', '4102', '4103', '4104', '4105', '4106', '4108']

    df_diagnoses_icd['icd_code_cleaned'] = df_diagnoses_icd['icd_code'].str.replace('.', '', regex=False).str.upper()

    mask_stemi = (
        (df_diagnoses_icd['icd_version'] == 9) &
        (df_diagnoses_icd['icd_code_cleaned'].str.startswith(tuple(stemi_icd9))) &
        (df_diagnoses_icd['icd_code_cleaned'].str.endswith('1'))
    ) | (
        (df_diagnoses_icd['icd_version'] == 10) &
        (df_diagnoses_icd['icd_code_cleaned'].str.startswith(tuple(stemi_icd10)))
    )

    stemi_diagnoses = df_diagnoses_icd[mask_stemi]
    stemi_admissions = stemi_diagnoses[['subject_id', 'hadm_id']].drop_duplicates()

    print(f"\n총 STEMI 진단 입원(hadm_id) 건수: {len(stemi_admissions)}")

    # [4] ED 방문 기록과 STEMI 입원 기록 병합 (코호트 생성)
    df_edstays_with_admission = df_edstays.dropna(subset=['hadm_id'])

    cohort_df = pd.merge(
        df_edstays_with_admission,
        stemi_admissions,
        on=['subject_id', 'hadm_id']
    )

    print("\n---  최종 코호트 생성 완료 ---")
    print(f"분석 대상 코호트 (ED -> STEMI 입원) 환자(stay_id) 수: {len(cohort_df)}")
    print(f"코호트 Shape: {cohort_df.shape}")
    print("\n[코호트 데이터 샘플 (상위 5건)]")
    print(cohort_df.head())


# [5] 오류 처리
except FileNotFoundError as e:
    print("\n" + "="*50)
    print(f"---  파일 로드 오류 (CSV) ---")
    print(f"오류: '{e.filename}' 파일을 찾을 수 없습니다.")
    print(f"경로: '{ed_base_path}'")
    print("팁: Colab 왼쪽 파일 탭에서 해당 경로에 파일이 있는지, 파일 이름이 정확한지(대소문자) 확인하세요.")
    print("="*50 + "\n")

except pd.io.sql.DatabaseError as e:
    print(f"--- DB 연결 오류 (SQLite) ---")
    print(f"오류: DB 연결이나 'diagnoses_icd' 테이블 쿼리에 실패했습니다.")
    print(f"경로: '{db_path}'")
    print(f"상세 오류: {e}")

except Exception as e:
    print(f"--- 알 수 없는 오류 ---")
    print(f"오류 발생: {e}")

edstays.csv 로드 완료 (Shape: (425087, 9))
diagnoses_icd DB 테이블 로드 완료 (Shape: (6364488, 4))

총 STEMI 진단 입원(hadm_id) 건수: 2533

---  최종 코호트 생성 완료 ---
분석 대상 코호트 (ED -> STEMI 입원) 환자(stay_id) 수: 836
코호트 Shape: (836, 9)

[코호트 데이터 샘플 (상위 5건)]
   subject_id   hadm_id   stay_id               intime              outtime  \
0    10010058  26359957  39919549  2147-11-18 00:50:00  2147-11-18 03:19:00   
1    10012438  22764825  35750515  2178-06-07 19:33:00  2178-06-07 21:57:00   
2    10013310  21243435  31736720  2153-05-26 08:56:00  2153-05-26 14:18:39   
3    10013310  27682188  35160955  2153-05-06 10:21:00  2153-05-06 18:28:00   
4    10033552  26487381  32542054  2136-02-29 22:09:00  2136-03-01 02:26:37   

  gender                      race arrival_transport disposition  
0      M  ASIAN - SOUTH EAST ASIAN         AMBULANCE    ADMITTED  
1      M                     WHITE         AMBULANCE    ADMITTED  
2      F             BLACK/AFRICAN           UNKNOWN    ADMITTED  
3      F             BLAC

#1.2 cohort 테이블 추출(2)
- mimic-iv-hosp-icu.db + mimic-iv-ed + mimic-iv-ecg
- ed에 방문해 stemi 진단을 받은 환자는 대부분 pci 시술을 했을 것이라고 성급하게 판단해 ecg 데이터를 결합하지 못했음 -> 1.2에서는 이를 보완

In [ ]:
# mimic-iv-ecg 데이터는 대용량(약 36GB)이므로 압축을 풀지 않고 파일 목록만 확인
# 상위 20개 파일 확인
!unzip -l "/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0.zip" | head -n 20

Archive:  /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
        0  2025-09-11 09:13   mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/
      299  2025-09-11 09:13   __MACOSX/._mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0
    13000  2023-08-31 06:05   mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/RECORDS
      299  2023-08-31 06:05   __MACOSX/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/._RECORDS
     6148  2025-09-11 09:57   mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/.DS_Store
      120  2025-09-11 09:57   __MACOSX/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/._.DS_Store
        0  2025-09-11 09:26   mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/
      299  2025-09-11 09:26   __MACOSX/mimic-iv-ecg-diagnostic-electrocardiogram-match

- 위의 파일 목록에서 확인된 ecg 메타데이터 파일인 record_list.csv 파일만 압축 해제하여 사용

In [ ]:
import os

zip_file_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0.zip'

# 압축 해제 경로 설정
extract_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'

# 압축 풀 파일의 정확한 전체 경로
file_to_extract = 'mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/record_list.csv'

!unzip -j -n "{zip_file_path}" "{file_to_extract}" -d "{extract_path}"

print("\n압축 해제 완료")

# 압축 해제된 파일 확인
print("\n저장된 파일 확인:")
!ls -lh "/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/record_list.csv"

Archive:  /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0.zip
  inflating: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/record_list.csv  
\n압축 해제 완료
\n저장된 파일 확인:
-rw------- 1 root root 68M Aug 31  2023 '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/record_list.csv'


In [13]:
ecg=pd.read_csv(ecg_csv_path, dtype={'subject_id': int, 'hadm_id': 'Int64'})
print(ecg.head())

   subject_id  study_id  file_name             ecg_time  \
0    10000032  40689238   40689238  2180-07-23 08:44:00   
1    10000032  44458630   44458630  2180-07-23 09:54:00   
2    10000032  49036311   49036311  2180-08-06 09:07:00   
3    10000117  45090959   45090959  2181-03-04 17:14:00   
4    10000117  48446569   48446569  2183-09-18 13:52:00   

                                       path  
0  files/p1000/p10000032/s40689238/40689238  
1  files/p1000/p10000032/s44458630/44458630  
2  files/p1000/p10000032/s49036311/49036311  
3  files/p1000/p10000117/s45090959/45090959  
4  files/p1000/p10000117/s48446569/48446569  


- 최종 코호트 추출 1단계: ecg 데이터 결합

In [18]:
import pandas as pd
import os
import sqlite3
import numpy as np

# [1] 경로 설정
ed_base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/mimic-iv-ed-2.2/'
db_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/MIMIC4-hosp-icu.db'
ecg_csv_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/record_list.csv'

# [2] 데이터 로드 및 코호트 생성
try:
    # 2-1. ED 데이터 로드 (CSV)
    # parse_dates를 추가하여 intime, outtime을 datetime 객체로 자동 변환
    edstays_csv_path = os.path.join(ed_base_path, 'edstays.csv')
    df_edstays = pd.read_csv(
        edstays_csv_path,
        dtype={'subject_id': int, 'hadm_id': 'Int64', 'stay_id': int},
        parse_dates=['intime', 'outtime']  # 시간 비교를 위해 datetime으로 로드
    )
    print(f"[2-1] edstays.csv 로드 완료 (Shape: {df_edstays.shape})")

    # 2-2. HOSP 데이터 로드 (SQLite)
    conn = sqlite3.connect(db_path) # DB 연결
    query = "SELECT subject_id, hadm_id, icd_code, icd_version FROM diagnoses_icd"
    df_diagnoses_icd = pd.read_sql_query(query, conn)
    conn.close() # DB 연결 해제

    df_diagnoses_icd['icd_code'] = df_diagnoses_icd['icd_code'].astype(str)
    print(f"[2-2] diagnoses_icd DB 테이블 로드 완료 (Shape: {df_diagnoses_icd.shape})")

    # [3] STEMI 진단 코드 정의 및 필터링
    stemi_icd10 = ['I210', 'I211', 'I212', 'I213']
    stemi_icd9 = ['4100', '4101', '4102', '4103', '4104', '4105', '4106', '4108']
    df_diagnoses_icd['icd_code_cleaned'] = df_diagnoses_icd['icd_code'].str.replace('.', '', regex=False).str.upper()

    mask_stemi = (
        (df_diagnoses_icd['icd_version'] == 9) &
        (df_diagnoses_icd['icd_code_cleaned'].str.startswith(tuple(stemi_icd9))) &
        (df_diagnoses_icd['icd_code_cleaned'].str.endswith('1'))
    ) | (
        (df_diagnoses_icd['icd_version'] == 10) &
        (df_diagnoses_icd['icd_code_cleaned'].str.startswith(tuple(stemi_icd10)))
    )

    stemi_diagnoses = df_diagnoses_icd[mask_stemi]
    stemi_admissions = stemi_diagnoses[['subject_id', 'hadm_id']].drop_duplicates()
    print(f"[3] 총 STEMI 진단 입원(hadm_id) 건수: {len(stemi_admissions)}")

    # [4] ED 방문 기록과 STEMI 입원 기록 병합 (HOSP + ED 코호트)
    df_edstays_with_admission = df_edstays.dropna(subset=['hadm_id'])
    cohort_df = pd.merge(
        df_edstays_with_admission,
        stemi_admissions,
        on=['subject_id', 'hadm_id'],
        how='inner'
    )
    print("[4] (HOSP + ED) 1차 코호트 생성 완료")
    print(f"분석 대상 코호트 (ED -> STEMI 입원) 환자(stay_id) 수: {len(cohort_df)}")
    print(f"코호트 Shape: {cohort_df.shape}")

    # (ECG 병합 로직)
    # [5] ECG 데이터 로드 및 시간 기반 최종 코호트 병합
    print("[5] ECG 데이터 로드 및 시간 기반 결합 시작")

    # 5-1. ECG 데이터 로드 (CSV)
    df_ecg = pd.read_csv(
        ecg_csv_path,
        dtype={'subject_id': int}, # 'hadm_id'가 없으므로 dtype에서 제거
        parse_dates=['ecg_time']   # 시간 비교를 위해 datetime으로 로드
    )
    # 필요한 컬럼만 선택함
    df_ecg = df_ecg[['subject_id', 'ecg_time', 'study_id']].dropna()
    print(f"record_list.csv 로드 및 전처리 완료 (Shape: {df_ecg.shape})")

    # 5-2. (HOSP+ED) 코호트와 ECG 기록을 'subject_id'로 병합
    print("1차 코호트와 ECG 기록을 'subject_id' 기준으로 병합")
    merged_cohort_ecg = pd.merge(
        cohort_df, # (subject_id, stay_id, hadm_id, intime, outtime, ...)
        df_ecg,    # (subject_id, ecg_time, study_id)
        on='subject_id', # 'hadm_id'가 없으므로 subject_id로만 병합
        how='inner'
    )
    print(f"Subject_id 기준 병합 완료 (Shape: {merged_cohort_ecg.shape})")

    # 5-3. 시간 필터링: ED 체류시간(intime ~ outtime) 내에 찍은 ECG만 필터링
    print("ED 체류 시간 내에 촬영된 ECG를 필터링")
    mask_time = (merged_cohort_ecg['ecg_time'] >= merged_cohort_ecg['intime']) & \
                (merged_cohort_ecg['ecg_time'] <= merged_cohort_ecg['outtime'])

    relevant_ecg_stays = merged_cohort_ecg[mask_time]
    print(f"ED 체류 중 ECG가 확인된 기록 수: {len(relevant_ecg_stays)}")

    # 5-4. ED 방문(stay_id)별 '첫 번째' ECG 시간(Door-to-ECG) 추출
    first_ecg_per_stay = relevant_ecg_stays.groupby('stay_id')['ecg_time'].min().reset_index()
    first_ecg_per_stay = first_ecg_per_stay.rename(columns={'ecg_time': 'first_ecg_time'})
    print(f"ED 체류 중 ECG가 확인된 고유 stay_id 수: {len(first_ecg_per_stay)}")

    # 5-5. 1차 코호트(cohort_df)와 'first_ecg_time'을 병합하여 최종 코호트 생성
    # how='inner'를 사용해 ED 체류 중 ECG 기록이 있는 환자(stay_id)만 남김
    final_cohort = pd.merge(
        cohort_df,
        first_ecg_per_stay,
        on='stay_id',
        how='inner'
    )

    # Door-to-ECG 시간(분) 계산
    final_cohort['door_to_ecg_min'] = (final_cohort['first_ecg_time'] - final_cohort['intime']).dt.total_seconds() / 60.0

    print("\n(HOSP + ED + ECG) 최종 코호트 생성 완료")
    print(f"분석 대상 최종 코호트 (ED -> STEMI 입원 + ED내 ECG) 환자(stay_id) 수: {len(final_cohort)}")
    print(f"최종 코호트 Shape: {final_cohort.shape}")
    print("[최종 코호트 데이터 샘플 (상위 5건)]")
    print(final_cohort.head())
    # (여기까지 ECG 병합)


# [6] 오류 처리
except FileNotFoundError as e:
    print("파일 로드 오류 (CSV)")
    print(f"오류: '{e}'")
    print("팁: 파일 경로에 파일이 있는지, 파일 이름이 정확한지(대소문자) 확인할 것.")

except pd.io.sql.DatabaseError as e:
    print("DB 연결 오류 (SQLite)")
    print(f"오류: DB 연결이나 테이블 쿼리에 실패함.")
    print(f"경로: '{db_path}'")
    print(f"상세 오류: {e}")

except Exception as e:
    print("알 수 없는 오류")
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc() # 오류의 상세 위치 추적을 위해 추가

[2-1] edstays.csv 로드 완료 (Shape: (425087, 9))
[2-2] diagnoses_icd DB 테이블 로드 완료 (Shape: (6364488, 4))
[3] 총 STEMI 진단 입원(hadm_id) 건수: 2533
[4] (HOSP + ED) 1차 코호트 생성 완료
분석 대상 코호트 (ED -> STEMI 입원) 환자(stay_id) 수: 836
코호트 Shape: (836, 9)
[5] ECG 데이터 로드 및 시간 기반 결합 시작
record_list.csv 로드 및 전처리 완료 (Shape: (800035, 3))
1차 코호트와 ECG 기록을 'subject_id' 기준으로 병합
Subject_id 기준 병합 완료 (Shape: (12904, 11))
ED 체류 시간 내에 촬영된 ECG를 필터링
ED 체류 중 ECG가 확인된 기록 수: 1466
ED 체류 중 ECG가 확인된 고유 stay_id 수: 708

(HOSP + ED + ECG) 최종 코호트 생성 완료
분석 대상 최종 코호트 (ED -> STEMI 입원 + ED내 ECG) 환자(stay_id) 수: 708
최종 코호트 Shape: (708, 11)
[최종 코호트 데이터 샘플 (상위 5건)]
   subject_id   hadm_id   stay_id              intime             outtime  \
0    10013310  21243435  31736720 2153-05-26 08:56:00 2153-05-26 14:18:39   
1    10013310  27682188  35160955 2153-05-06 10:21:00 2153-05-06 18:28:00   
2    10033552  26487381  32542054 2136-02-29 22:09:00 2136-03-01 02:26:37   
3    10055344  29209451  39410952 2171-10-31 14:50:00 2171-10-31 19:38:25   
4

In [19]:
# [7] PCI 시술 데이터 결합 (HOSP + ED + ECG + PCI)
# 이전 셀의 'final_cohort' 변수가 메모리에 존재한다고 가정함.

print("[7] PCI 시술 데이터 로드 및 결합 시작")

# 7-1. PCI 시술에 해당하는 ICD 코드 정의
# (ICD-9-CM 및 ICD-10-PCS 기준)

# ICD-9 시술 코드
pci_icd9_codes = [
    '0066', # PTCA (경피적 관상동맥 확장술)
    '3601', # 1개 혈관 PTCA
    '3602', # 1개 혈관 PTCA (약물방출 스텐트 없음)
    '3605', # 다중 혈관 PTCA
    '3606', # 관상동맥 스텐트 삽입
    '3607', # 관상동맥 스텐트 삽입 (약물방출)
    '3609'  # 기타 관상동맥 혈관성형술
]

# ICD-10 시술 코드 (PCS) - '관상동맥 확장(Dilation)' 관련
pci_icd10_prefixes = [
    '0270', # 관상동맥, 1개 부위
    '0271', # 관상동맥, 2개 부위
    '0272', # 관상동맥, 3개 부위
    '0273'  # 관상동맥, 4개 이상 부위
]

try:
    # 7-2. 변수 존재 여부 확인
    if 'final_cohort' not in locals() or 'db_path' not in locals():
        print("오류: 'final_cohort' 또는 'db_path' 변수가 없습니다.")
        print("이전 셀([1]~[5])을 먼저 실행해야 함.")
    else:
        # 7-3. HOSP DB에서 'procedures_icd' 테이블 로드
        print("HOSP DB에서 'procedures_icd' 테이블을 로드함...")
        conn = sqlite3.connect(db_path)
        query_proc = "SELECT subject_id, hadm_id, icd_code, icd_version FROM procedures_icd"
        df_procedures_icd = pd.read_sql_query(query_proc, conn)
        conn.close()

        # ICD 코드 전처리
        df_procedures_icd['icd_code'] = df_procedures_icd['icd_code'].astype(str).str.replace('.', '', regex=False).str.upper()
        print(f"procedures_icd 로드 완료 (Shape: {df_procedures_icd.shape})")

        # 7-4. PCI 시술 코드 필터링
        mask_pci_9 = (
            (df_procedures_icd['icd_version'] == 9) &
            (df_procedures_icd['icd_code'].isin(pci_icd9_codes))
        )

        mask_pci_10 = (
            (df_procedures_icd['icd_version'] == 10) &
            (df_procedures_icd['icd_code'].str.startswith(tuple(pci_icd10_prefixes)))
        )

        pci_procedures = df_procedures_icd[mask_pci_9 | mask_pci_10]

        # 7-5. PCI 시술을 받은 입원(hadm_id) 목록 추출
        pci_admissions = pci_procedures[['subject_id', 'hadm_id']].drop_duplicates().dropna()
        print(f"PCI 시술 기록이 있는 고유 입원(hadm_id) 건수: {len(pci_admissions)}")

        # 7-6. (HOSP+ED+ECG) 코호트와 (PCI) 시술 목록 병합
        final_cohort_with_pci = pd.merge(
            final_cohort,    # (HOSP + ED + ECG) 코호트 (708건)
            pci_admissions,  # PCI 시술을 받은 입원 목록
            on=['subject_id', 'hadm_id'],
            how='inner'      # (HOSP ∩ ED ∩ ECG ∩ PCI)
        )

        print("\n(HOSP + ED + ECG + PCI) 최종 코호트 생성 완료")
        print(f"분석 대상 최종 코호트 환자(stay_id) 수: {len(final_cohort_with_pci)}")
        print(f"최종 코호트 Shape: {final_cohort_with_pci.shape}")
        print("\n[최종 코호트(PCI포함) 데이터 샘플 (상위 5건)]")
        print(final_cohort_with_pci.head())

# 7-7. 오류 처리
except NameError as e:
    print(f"오류: 필요한 변수가 정의되지 않음: {e}")
except pd.io.sql.DatabaseError as e:
    print("DB 연결 오류 (SQLite)")
    print(f"오류: DB 연결이나 'procedures_icd' 테이블 쿼리에 실패함.")
    print(f"경로: '{db_path}'")
    print(f"상세 오류: {e}")
except Exception as e:
    print("알 수 없는 오류")
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc() # 오류의 상세 위치 추적을 위해 추가함

[7] PCI 시술 데이터 로드 및 결합 시작
HOSP DB에서 'procedures_icd' 테이블을 로드함...
procedures_icd 로드 완료 (Shape: (859655, 4))
PCI 시술 기록이 있는 고유 입원(hadm_id) 건수: 8075

(HOSP + ED + ECG + PCI) 최종 코호트 생성 완료
분석 대상 최종 코호트 환자(stay_id) 수: 465
최종 코호트 Shape: (465, 11)

[최종 코호트(PCI포함) 데이터 샘플 (상위 5건)]
   subject_id   hadm_id   stay_id              intime             outtime  \
0    10013310  27682188  35160955 2153-05-06 10:21:00 2153-05-06 18:28:00   
1    10033552  26487381  32542054 2136-02-29 22:09:00 2136-03-01 02:26:37   
2    10055344  29209451  39410952 2171-10-31 14:50:00 2171-10-31 19:38:25   
3    10096420  25396519  39083351 2204-07-16 12:11:00 2204-07-16 13:58:56   
4    10130348  24286651  38674082 2197-05-27 06:54:00 2197-05-27 10:41:05   

  gender                    race arrival_transport disposition  \
0      F           BLACK/AFRICAN         AMBULANCE    ADMITTED   
1      F                   OTHER         AMBULANCE    ADMITTED   
2      M      BLACK/CAPE VERDEAN           UNKNOWN       OTHER   
3 

In [20]:
# [8] 최종 코호트 검토 (결측치, 이상치)
# 이전 셀의 'final_cohort_with_pci' 변수가 메모리에 존재한다고 가정함.

print("[8] 최종 코호트(465건) 결측치 및 이상치 검토 시작")

try:
    # 8-1. 변수 존재 여부 확인
    if 'final_cohort_with_pci' not in locals():
        print("오류: 'final_cohort_with_pci' 변수가 없습니다.")
        print("이전 셀([1]~[7])을 먼저 실행해야 함.")
    else:
        # 8-2. 결측치(Missing Values) 확인
        # 각 컬럼별로 null/NaN 값이 몇 개인지 합산하여 출력함
        print("\n[8-1] 컬럼별 결측치 개수 확인")
        print(final_cohort_with_pci.isnull().sum())

        # 8-3. 이상치(Outliers) 확인 (수치형 데이터 요약)
        # .describe()로 숫자 컬럼의 통계 (min, max, mean, 4분위수)를 확인
        # 특히 'door_to_ecg_min' 컬럼을 중점적으로 확인
        print("\n[8-2] 'door_to_ecg_min' 컬럼 통계 요약 (이상치 확인용)")
        print(final_cohort_with_pci['door_to_ecg_min'].describe())

        # (참고) 'door_to_ecg_min' 값이 음수(-)이거나 비정상적으로 큰(e.g., 1000분 이상)
        # 데이터가 있는지 min/max 값을 통해 확인할 수 있음.

except Exception as e:
    print("알 수 없는 오류")
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc() # 오류의 상세 위치 추적을 위해 추가함

[8] 최종 코호트(465건) 결측치 및 이상치 검토 시작

[8-1] 컬럼별 결측치 개수 확인
subject_id           0
hadm_id              0
stay_id              0
intime               0
outtime              0
gender               0
race                 0
arrival_transport    0
disposition          0
first_ecg_time       0
door_to_ecg_min      0
dtype: int64

[8-2] 'door_to_ecg_min' 컬럼 통계 요약 (이상치 확인용)
count    465.000000
mean      17.290323
std       32.386733
min        0.000000
25%        7.000000
50%       10.000000
75%       14.000000
max      430.000000
Name: door_to_ecg_min, dtype: float64


In [21]:
# [9] 최종 코호트 CSV 파일 저장
# 이전 셀에서 'final_cohort_with_pci' 변수가 생성되었다고 가정함.

print("\n[9] 최종 코호트 파일 저장 시작")

try:
    # 9-1. 변수 존재 여부 확인
    if 'final_cohort_with_pci' not in locals():
        print("오류: 'final_cohort_with_pci' 변수가 없습니다.")
        print("이전 셀([1]~[8])을 먼저 실행해야 함.")
    else:
        # 9-2. CSV 파일로 저장
        save_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/cohort.csv'

        # index=False: pandas의 불필요한 인덱스(0, 1, 2...)를 CSV에 저장하지 않음
        # encoding='utf-8-sig': Excel에서 열었을 때 한글 깨짐을 방지함
        final_cohort_with_pci.to_csv(save_path, index=False, encoding='utf-8-sig')

        print(f"\n[9-1] 최종 코호트(Shape: {final_cohort_with_pci.shape})를 파일로 저장 완료")
        print(f"저장 경로: {save_path}")

# 9-3. 오류 처리
except Exception as e:
    print("알 수 없는 오류")
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc() # 오류의 상세 위치 추적을 위해 추가함


[9] 최종 코호트 파일 저장 시작

[9-1] 최종 코호트(Shape: (465, 11))를 파일로 저장 완료
저장 경로: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/cohort.csv
